https://www.blog.pythonlibrary.org/2018/06/07/an-intro-to-pypdf2/

import glob

for name in glob.glob('2019CBC-11B-*.pdf'):
    print(name)

# Libraries

In [232]:
from PyPDF2 import PdfFileReader
import re
import pandas as pd

# Functions

In [233]:
def PDF2TXT(filename):
    with open(filename, "rb") as filehandle:
        pdf = PdfFileReader(filehandle)
        info = pdf.getDocumentInfo()
        num_pages = pdf.getNumPages()
        # print(info) # Document Information
        # print("Number of pages: %i" % pages) # Print total number of pages

    ### Test for one page to be extracted
        # page1 = pdf.getPage(0) # Work with page 0
        # print(page1)
        # pg_extracted = page1.extractText()
        # print(pg_extracted)
        # cleaned_pg = pg_extracted.replace("11B-\n*", "\n11B-*").replace("Section \n11B-", "Section 11B-")
        #concat = False

    ### All pages are extracted
        text_extracted=''
        for num in range(num_pages):
            pg = pdf.getPage(num)
            txt_extracted = pg.extractText()
            text_extracted += txt_extracted
    return text_extracted

In [234]:
def SplitText(text_extracted):
    text = ""
    # for line in pg_extracted.splitlines():
    for line in text_extracted.splitlines():
        text += line + " "

    # paragraph break
    newline = re.compile(r'(11B-)')

    new_text = ""
    begin = 0
    _end = 0
    for m in newline.finditer(text):
        begin, end = m.span()
        if text[begin-10:begin].endswith('Section '): 
            continue
        new_text += text[_end:begin] + "\n\n"
        _end = begin

    new_text += text[_end:]

    # remove hyphen
    hyphene1 = re.compile(r'([A-Za-z]+)- ([a-z]+)')
    new_text = hyphene1.sub(r'\1\2', new_text)
    hyphene2 = re.compile(r'([A-Za-z]+)-([a-z]+)')
    new_text = hyphene2.sub(r'\1\2', new_text)

    # remove units
    units = re.compile(r' \((\s+)?[0-9]+\s+?mm(\s+)?\)')
    new_text = units.sub('', new_text)

    # remove header and footer
    new_text = new_text.replace('ACCESSIBILITY TO PUBLIC BUILDINGS, PUBLIC ACCOMMODATIONS, COMMERCIAL BUILDINGS AND PUBLIC HOUSING', '')
    new_text = new_text.replace('ACCESSIBILITY TO PUBLIC BUILDINGS, PUBLIC ACCOMMO DATIONS, COMMERCIAL BUILDINGS AND PUBLIC HOUSING', '')
    footer1 = re.compile(r'2016 CALIFORNIA BUILDING CODE \d+')
    new_text = footer1.sub('', new_text)
    footer2 = re.compile(r'\d+2016 CALIFORNIA BUILDING CODE ')
    new_text = footer2.sub('', new_text)

    # remove white space
    new_text = new_text.replace('11B- ', '11B-')
    new_text = new_text.replace('  ', ' ')
    new_text = new_text.replace(' .', '.')

    # remove new line
    new_text = new_text.replace("FIGURE \n\n11B-", "\nFIGURE 11B-")
    new_text = new_text.replace(". Exception:", ". \nException:")
    new_text = new_text.replace("or \n\n11B-", "or 11B-")
    new_text = new_text.replace(", \n\n11B-", ", 11B-")
    new_text = new_text.replace("and \n\n11B-", "and 11B-")
    new_text = new_text.replace("with Section \n\n11B-", "with Section 11B-")
    new_text = new_text.replace("with Sections \n\n11B-", "with Sections 11B-")

    # prep str by splitting by delimiter
    new_text = new_text.replace('\n\n', ',,,')

    split_text = new_text.split(",,,")
    return split_text


In [235]:
# list to dictionary
def Convert2dict(lst): 
    length = len(lst)
    result = {i: lst[i] for i in range(length)}
    return result

#########
# https://www.kite.com/python/answers/how-to-parse-a-csv-string-in-python
# import csv
# lines = new_text.splitlines()
# reader = csv.reader(lines)
# reader#parsed_csv = list(reader)
#########


In [236]:
def SaveDictToCsv (dict_text, csv_filename):
    df = pd.DataFrame.from_dict(dict_text, orient='index')
    # df.head()
    df.to_csv(csv_filename, sep=',',index=False)


# Action

In [238]:
# filename = 'test.pdf'
# filename = '2016CBC-161-165.pdf'

for num in range(10):
    filename = '2019CBC-11B-%s.pdf'%(num+1)
    text_extracted = PDF2TXT(filename)
    split_text = SplitText(text_extracted)
    dict_text = Convert2dict(split_text)
    csv_filename = '2019CBC-11B-Section%s.csv'%(num+1)
    SaveDictToCsv(dict_text, csv_filename)